In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import pydub
import json

## 处理 song_info

In [2]:
song_info = pd.read_csv('song_info.csv', encoding='utf8')
song_info.head()

,song,lyricist,composer,arranger
0,一路向北,方文山,周杰伦,蔡科俊
1,发如雪,方文山,周杰伦,NaN
2,四面楚歌,周杰伦,周杰伦,NaN
3,夜曲,方文山,周杰伦,NaN
4,枫,宋健彰(弹头),周杰伦,NaN


In [3]:
song_info.shape

(170, 4)

In [4]:
song_info.replace({'lyricist': {'古小力、黄凌嘉': '古小力/黄凌嘉', '宋健彰(弹头)': '宋健彰', '方文山/ST': '方文山', '罗宇轩、李汪哲': '罗宇轩/李汪哲', '罗宇轩、黄婕熙': '罗宇轩/黄婕熙'}, 
                  'composer': {'周杰伦、Hathaway': '周杰伦/Hathaway'}, 
                  'arranger': {'蔡科俊Again': '蔡科俊', '黄雨勋(梦想之翼)': '黄雨勋'}},
                  inplace=True)
song_info.sample(5)

,song,lyricist,composer,arranger
14,借口,周杰伦,周杰伦,周杰伦
15,园游会,方文山,周杰伦,洪敬尧
63,你听得到,曾郁婷,周杰伦,林迈可
134,斗牛,方文山,周杰伦,洪敬尧
96,你听得到(Vcd),NaN,NaN,NaN


In [5]:
song_info.shape

(170, 4)

In [6]:
song_info.set_index('song', drop=True, inplace=True)
song_info.sample(5)

,lyricist,composer,arranger
song,,,
免费教学录影带,黄俊郎,周杰伦,蔡科俊
分裂,周杰伦,周杰伦,钟兴民
夜的第七章电影版MV(DVD),黄俊郎,周杰伦,NaN
稻香,周杰伦,周杰伦,NaN
可爱女人,徐若瑄,周杰伦,周杰伦


In [7]:
song_info.shape

(170, 3)

In [8]:
song_info.index = [idx.strip() for idx in song_info.index]
song_info.sample(5)

,lyricist,composer,arranger
傻笑(& 袁咏琳),方文山,周杰伦,黄雨勋
彩虹,周杰伦,周杰伦,NaN
双刀(Vcd),NaN,NaN,NaN
甜甜的,方文山,周杰伦,NaN
夜的第七章,黄俊郎,周杰伦,NaN


In [9]:
def bad_name(name):
    for ch in ['original karaoke', 'dvd', 'vcd', 'mtv', '电影版', '完整版']:
        if ch in name.lower():
            return True
    else:
        return False

In [10]:
drop_them = [idx for idx in song_info.index if bad_name(idx)]
drop_them

['千里之外电影版MV(DVD)',
 '夜的第七章电影版MV(DVD)',
 '最后的战役(完整版Mtv)',
 '最后的战役(电影版Mtv)',
 '三年二班(Vcd)',
 '东风破(Vcd)',
 '他的睫毛(Vcd)',
 '以父之名(Vcd)',
 '你听得到(Vcd)',
 '双刀(Vcd)',
 '同一种调调(Vcd)',
 '懦夫(Vcd)',
 '断了的弦(Original Karaoke)',
 '晴天(Vcd)',
 '梯田(Vcd)',
 '爱情悬崖(Vcd)',
 '轨迹(Original Karaoke)']

In [11]:
len(drop_them)

17

In [12]:
song_info.drop(drop_them, axis=0, inplace=True)
song_info.sample(5)

,lyricist,composer,arranger
你听得到,曾郁婷,周杰伦,林迈可
困兽之斗,刘畊宏,周杰伦,蔡科俊
蓝色风暴,方文山,周杰伦,NaN
我不配,方文山,周杰伦,NaN
美人鱼,罗宇轩/黄婕熙,周杰伦,林迈可


In [13]:
song_info.shape

(153, 3)

In [14]:
song_info.lyricist.unique()

array(['方文山', '周杰伦', '宋健彰', '黄俊郎', '刘畊宏', '许世昌', '林义杰', '黄凌嘉', '曾郁婷',
       '徐若瑄', '罗宇轩/黄婕熙', '罗宇轩/李汪哲', '唐从圣', '古小力/黄凌嘉'], dtype=object)

In [15]:
song_info.composer.unique()

array(['周杰伦', '周杰伦/Hathaway'], dtype=object)

In [16]:
song_info.arranger.unique()

array(['蔡科俊', nan, '钟兴民', '林迈可', '周杰伦', '洪敬尧', '迈可林', '黄雨勋', '蔡庭贵'],
      dtype=object)

In [17]:
song_info.arranger.isnull().sum()

44

## 处理歌词

In [18]:
files = Path('Lyrics/').glob('**/*.txt')
d = {}
for file in files:
    print(file)
    name = file.stem
    if bad_name(name):
        continue
    lyric = file.read_text(encoding='utf8')
    d[name] = lyric

Lyrics\11月的萧邦\一路向北.txt
Lyrics\11月的萧邦\发如雪.txt
Lyrics\11月的萧邦\四面楚歌.txt
Lyrics\11月的萧邦\夜曲.txt
Lyrics\11月的萧邦\枫.txt
Lyrics\11月的萧邦\浪漫手机.txt
Lyrics\11月的萧邦\珊瑚海(&梁心颐).txt
Lyrics\11月的萧邦\蓝色风暴.txt
Lyrics\11月的萧邦\逆鳞.txt
Lyrics\11月的萧邦\飘移.txt
Lyrics\11月的萧邦\麦芽糖.txt
Lyrics\11月的萧邦\黑色毛衣.txt
Lyrics\七里香\七里香.txt
Lyrics\七里香\乱舞春秋.txt
Lyrics\七里香\借口.txt
Lyrics\七里香\园游会.txt
Lyrics\七里香\困兽之斗.txt
Lyrics\七里香\外婆.txt
Lyrics\七里香\将军.txt
Lyrics\七里香\我的地盘.txt
Lyrics\七里香\搁浅.txt
Lyrics\七里香\止战之殇.txt
Lyrics\不能说的秘密\不能说的秘密.txt
Lyrics\依然范特西\千里之外(&费玉清).txt
Lyrics\依然范特西\千里之外电影版MV(DVD).txt
Lyrics\依然范特西\听妈妈的话.txt
Lyrics\依然范特西\夜的第七章.txt
Lyrics\依然范特西\夜的第七章电影版MV(DVD).txt
Lyrics\依然范特西\心雨.txt
Lyrics\依然范特西\本草纲目.txt
Lyrics\依然范特西\白色风车.txt
Lyrics\依然范特西\红模仿.txt
Lyrics\依然范特西\菊花台.txt
Lyrics\依然范特西\迷迭香.txt
Lyrics\依然范特西\退后.txt
Lyrics\八度空间\分裂.txt
Lyrics\八度空间\半兽人.txt
Lyrics\八度空间\半岛铁盒.txt
Lyrics\八度空间\回到过去.txt
Lyrics\八度空间\暗号.txt
Lyrics\八度空间\最后的战役(完整版Mtv).txt
Lyrics\八度空间\最后的战役(电影版Mtv).txt
Lyrics\八度空间\最后的战役.txt
Lyrics\八度空间\火车叨位去.txt
Lyrics\八度空间\爷爷泡的茶.txt
Lyr

In [19]:
lyrics = pd.DataFrame(list(d.items()), columns=['song', 'lyric'])
lyrics.set_index('song', drop=True, inplace=True)
lyrics.head()

,lyric
song,
一路向北,头文字D插曲\n作词：方文山\n作曲：周杰伦\n编曲：蔡科俊\n后视镜里的世界 越来越远的道...
发如雪,作词：方文山\n作曲：周杰伦\n狼牙月 伊人憔悴\n我举杯 饮尽了风雪\n是谁打翻前世柜 ...
四面楚歌,作词：周杰伦\n作曲：周杰伦\n我的生活像拍了一出戏\n有超多导演跟编剧\n只说了台词一句\...
夜曲,作词：方文山\n作曲：周杰伦\n一群嗜血的蚂蚁 被腐肉所吸引 我面无表情 看孤独的风景\n失...
枫,作词：宋健彰(弹头)\n作曲：周杰伦\n乌云在我们心里搁下一块阴影\n我聆听沉寂已久的心情\...


In [20]:
lyrics.shape

(153, 1)

## 合并歌词和歌曲信息

In [21]:
song_info = pd.merge(song_info, lyrics, left_index=True, right_index=True)
song_info.sample(5)

,lyricist,composer,arranger,lyric
song,,,,
夜的第七章,黄俊郎,周杰伦,NaN,作词：黄俊郎\n作曲：周杰伦\n女声：潘儿(Room19)\n1983年小巷 12月晴朗 夜...
烟花易冷,方文山,周杰伦,黄雨勋,作词：方文山\n作曲：周杰伦\n编曲：黄雨勋(梦想之翼)\n作词：方文山\n作曲：周杰伦\n...
蓝色风暴,方文山,周杰伦,NaN,作词：方文山\n作曲：周杰伦\n亘古长夜 善恶交接 我终于懂得流眼泪\n洪荒世界 百分之七十...
算什么男人,周杰伦,周杰伦,黄雨勋,作词：周杰伦\n作曲：周杰伦\n编曲：黄雨勋\n亲吻你的手 还靠着你的头\n让你躺胸口 那个...
千里之外(&费玉清),方文山,周杰伦,NaN,作词：方文山\n作曲：周杰伦\n周：屋簷如悬崖 风铃如沧海 我等燕归来\n时间被安排 演一场...


## 处理歌曲元数据

In [22]:
song_metadata = pd.read_csv('all_jay_songs.csv')
song_metadata.head()

,DISPOSITION_attached_pic,DISPOSITION_clean_effects,DISPOSITION_comment,DISPOSITION_default,DISPOSITION_dub,DISPOSITION_forced,DISPOSITION_hearing_impaired,DISPOSITION_karaoke,DISPOSITION_lyrics,DISPOSITION_original,...,TAG_mp3gain_minmax,TAG_replaygain_track_gain,TAG_replaygain_track_peak,TAG_ENCODEDBY,TAG_PLAY_COUNTER,TAG_PLAY_DATE,TAG_PLAY_TIME,TAG_album_artist,TAG_mp3gain_undo,TAG_WWW
0,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# 去掉全 0 列、全 1 列和全 nan 列
song_metadata = song_metadata.loc[:, (song_metadata != 0).any()]
song_metadata = song_metadata.loc[:, (song_metadata != 1).any()]
song_metadata.dropna(axis=1, how='all', inplace=True)
song_metadata.head()

,DISPOSITION_attached_pic,TAG_album,TAG_artist,TAG_encoder,TAG_genre,TAG_title,TAG_track,avg_frame_rate,bit_rate,bits_per_raw_sample,...,TAG_mp3gain_minmax,TAG_replaygain_track_gain,TAG_replaygain_track_peak,TAG_ENCODEDBY,TAG_PLAY_COUNTER,TAG_PLAY_DATE,TAG_PLAY_TIME,TAG_album_artist,TAG_mp3gain_undo,TAG_WWW
0,1,Jay,周杰伦,LAME3.97,Other,可爱女人.mp3,1.0,0/0,321289,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Jay,周杰伦,LAME3.97,Other,完美主义.mp3,2.0,0/0,321260,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Jay,周杰伦,LAME3.97,Other,星晴.mp3,3.0,0/0,321188,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Jay,周杰伦,LAME3.97,Other,娘子.mp3,4.0,0/0,321137,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Jay,周杰伦,LAME3.97,Other,斗牛.mp3,5.0,0/0,321105,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
song_metadata.TAG_title = song_metadata.TAG_title.apply(lambda t: Path(t).stem)
song_metadata.TAG_title.head()

0    可爱女人
1    完美主义
2      星晴
3      娘子
4      斗牛
Name: TAG_title, dtype: object

In [25]:
# 重新设置歌名为 index，扔掉 bad name
song_metadata.rename(columns={'TAG_title': 'song'}, inplace=True)
song_metadata.set_index('song', drop=True, inplace=True)
drop_them = [idx for idx in song_metadata.index if bad_name(idx)]
song_metadata.drop(drop_them, axis=0, inplace=True)
song_metadata.sample(5)

,DISPOSITION_attached_pic,TAG_album,TAG_artist,TAG_encoder,TAG_genre,TAG_track,avg_frame_rate,bit_rate,bits_per_raw_sample,channel_layout,...,TAG_mp3gain_minmax,TAG_replaygain_track_gain,TAG_replaygain_track_peak,TAG_ENCODEDBY,TAG_PLAY_COUNTER,TAG_PLAY_DATE,TAG_PLAY_TIME,TAG_album_artist,TAG_mp3gain_undo,TAG_WWW
song,,,,,,,,,,,,,,,,,,,,,
The Swan,1,不能说的秘密,周杰伦,NaN,NaN,NaN,0/0,322819,8.0,stereo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鞋子特大号,0,哎呦，不错哦,周杰伦,LAME3.99r,NaN,9.0,0/0,320119,NaN,stereo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
半兽人,1,八度空间,周杰伦,NaN,Pop,NaN,0/0,321379,8.0,stereo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
借口,1,七里香,周杰伦,NaN,R&B,3.0,0/0,320739,8.0,stereo,...,"086,210",-8.310000 dB,1.026755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
退后,1,依然范特西,周杰伦,NaN,R&B,5.0,0/0,321686,8.0,stereo,...,NaN,NaN,NaN,iTunes v6.0.5.20,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
song_metadata.shape

(170, 71)

In [27]:
# 周杰伦的床边故事
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

df = pd.DataFrame()
for song in glob.glob('Jay Chou\周杰伦的床边故事\*.mp3', recursive=False):
    mediainfo = pydub.utils.mediainfo(song)
    df = df.append(flatten(mediainfo), ignore_index=True)

df.rename(columns={'TAG_title': 'song'}, inplace=True)
df.song = df.song.apply(lambda t: t.strip())
df.set_index('song', drop=True, inplace=True)
df.rename(index={'不该(with aMEI)': '不该'}, inplace=True)
df.head()

,DISPOSITION_attached_pic,DISPOSITION_clean_effects,DISPOSITION_comment,DISPOSITION_default,DISPOSITION_dub,DISPOSITION_forced,DISPOSITION_hearing_impaired,DISPOSITION_karaoke,DISPOSITION_lyrics,DISPOSITION_original,...,refs,sample_aspect_ratio,sample_fmt,sample_rate,size,start_pts,start_time,time_base,timecode,width
song,,,,,,,,,,,,,,,,,,,,,
Now You See Me,1,0,0,0,0,0,0,0,0,0,...,1,1:1,s16p,44100,7197241,2255,0.025056,1/90000,N/A,640
一点点,1,0,0,0,0,0,0,0,0,0,...,1,1:1,s16p,44100,9086330,2255,0.025056,1/90000,N/A,640
前世情人,1,0,0,0,0,0,0,0,0,0,...,1,1:1,s16p,44100,8263997,2255,0.025056,1/90000,N/A,640
告白气球,1,0,0,0,0,0,0,0,0,0,...,1,1:1,s16p,44100,8841826,2255,0.025056,1/90000,N/A,640
土耳其冰淇淋,1,0,0,0,0,0,0,0,0,0,...,1,1:1,s16p,44100,8065471,2255,0.025056,1/90000,N/A,640


In [28]:
# 去掉多出来的列
song_metadata = song_metadata.append(df).drop(set(df.columns) - set(song_metadata.columns), axis=1)
song_metadata.shape

(180, 71)

## 合并歌曲信息和歌曲元数据

In [29]:
set(song_metadata.index) - set(song_info.index)

{'Angel',
 'Ending',
 'First Kiss',
 'Flash Back',
 'Opening',
 'Ride With Me',
 'Secret (加长快板)',
 'Secret (慢板)',
 'The Swan',
 '上海 一九四三',
 '不该',
 '与父共舞',
 '世界末末日',
 '傻笑',
 '免费教学录像带',
 '分裂 离开',
 '千里之外',
 '女孩别为我哭泣',
 '她的睫毛',
 '小雨写立可白Ⅰ',
 '小雨写立可白Ⅱ',
 '情人的眼泪',
 '我落泪 情绪零碎',
 '我要夏天',
 '斗琴',
 '早操',
 '晴天娃娃',
 '比较大的大提琴',
 '淡水海边',
 '湘伦小雨四手联弹',
 '父与子',
 '爸 我回來了',
 '珊瑚海',
 '琴房',
 '紅模仿',
 '脚踏车',
 '菊花台 钢琴曲',
 '路小雨',
 '阿郎与阿宝',
 '霍元甲',
 '黄金甲 EP'}

In [30]:
set(song_info.index) - set(song_metadata.index)

{'上海一九四三',
 '不该(& 张惠妹)',
 '世界未末日',
 '他的睫毛',
 '傻笑(& 袁咏琳)',
 '免费教学录影带',
 '分裂',
 '千里之外(&费玉清)',
 '我落泪。情绪零碎',
 '我要夏天(& 杨瑞代)',
 '比较大的大提琴(&梁心颐&杨瑞代)',
 '爸我回来了',
 '珊瑚海(&梁心颐)',
 '红模仿'}

In [31]:
song_info.rename(index={
    '不该(& 张惠妹)': '不该', 
    '傻笑(& 袁咏琳)': '傻笑', 
    '千里之外(&费玉清)': '千里之外', 
    '我要夏天(& 杨瑞代)': '我要夏天', 
    '比较大的大提琴(&梁心颐&杨瑞代)': '比较大的大提琴', 
    '珊瑚海(&梁心颐)': '珊瑚海',
    '我落泪。情绪零碎': '我落泪情绪零碎',
    '他的睫毛': '她的睫毛'
}, inplace=True)

song_metadata.rename(index={
    '我落泪 情绪零碎': '我落泪情绪零碎',
    '世界末末日': '世界未末日',
    '分裂 离开': '分裂',
    '爸 我回來了': '爸我回来了',
    '紅模仿': '红模仿',
    '免费教学录像带': '免费教学录影带'
}, inplace=True)

In [32]:
song_info.sample(5)

,lyricist,composer,arranger,lyric
song,,,,
最后的战役,方文山,周杰伦,钟兴民,作词：方文山\n作曲：周杰伦\n编曲：钟兴民\n机枪扫射声中我们寻找遮蔽的战壕\n儿时沙雕的...
皮影戏,唐从圣,周杰伦,林迈可,作词：唐从圣\n作曲：周杰伦\n编曲：林迈可\n监制：周杰伦\n微薄的身躯 刻画出厚实尊严\...
比较大的大提琴,方文山,周杰伦,黄雨勋,作词：方文山\n作曲：周杰伦\n编曲：黄雨勋\n制作人：周杰伦\n小傻瓜这不是大提琴 我拨弦...
甜甜的,方文山,周杰伦,NaN,作词：方文山\n作曲：周杰伦\n作词：方文山\n作曲：周杰伦\n我轻轻的尝一口 妳说的爱我 ...
简单爱,徐若瑄,周杰伦,迈可林,民视Star Blue蓝星主题曲\n作词：徐若瑄\n作曲：周杰伦\n编曲：迈可林\n说不上为...


In [33]:
song_metadata.sample(5)

,DISPOSITION_attached_pic,TAG_AlbumArtist,TAG_ENCODEDBY,TAG_PLAY_COUNTER,TAG_PLAY_DATE,TAG_PLAY_TIME,TAG_WM/MediaPrimaryClassID,TAG_WM/Provider,TAG_WM/ProviderStyle,TAG_WM/Publisher,...,refs,sample_aspect_ratio,sample_fmt,sample_rate,side_data_type,size,start_pts,start_time,time_base,width
song,,,,,,,,,,,,,,,,,,,,,
魔术先生,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1:1,s16p,44100,Replay Gain,9166397,2255,0.025056,1/90000,357
扯,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0:1,s16p,44100,NaN,7510031,2255,0.025056,1/90000,352
黑色毛衣,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1:1,s16p,44100,NaN,10113346,NaN,0,1/90000,550
脚踏车,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1:1,s16p,44100,NaN,5536601,NaN,0,1/90000,240
花海,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1:1,s16p,44100,Replay Gain,10644924,2255,0.025056,1/90000,357


In [34]:
print(f'song_info shape: {song_info.shape}\nsong_metadata shape: {song_metadata.shape}')

song_info shape: (153, 4)
song_metadata shape: (180, 71)


In [35]:
song_info = pd.merge(song_metadata, song_info, left_index=True, right_index=True, how='left')
song_info.shape

(180, 75)

In [36]:
song_info.lyric.sample(5)

song
龙战骑士       作词：方文山\n作曲：周杰伦\n监制：周杰伦\n放手一搏令谁都惭愧 迎著风极速在超越\n那守...
阳光宅男       作词：方文山\n作曲：周杰伦\n钥匙挂腰带 皮夹插 后面口袋 黑框的眼镜有 几千度 来海边穿...
小雨写立可白Ⅱ                                                  NaN
困兽之斗       作词：刘畊宏\n作曲：周杰伦\n编曲：蔡科俊Again\n＊我在阴暗中降落 世界在雨中淹没\...
火车叨位去      作词：方文山\n作曲：周杰伦\n编曲：洪敬尧\n听我讲 庄脚ㄟ路边 蝉躲在树枝\n七月天 闹...
Name: lyric, dtype: object

In [37]:
s = '''作词：方文山
作曲：周杰伦

吓 命有几回合 擂台等著 生死状 赢了什么 冷笑着
天下谁的 第一又如何 止干戈 我辈尚武德
我的 拳脚了得 却奈何 徒增虚名一个
江湖难测 谁是强者 谁争一统武林的资格

小城里岁月流过去 清澈的勇气
洗涤过的回忆 我记得你 骄傲的活下去

霍霍 霍霍霍 霍霍霍 霍家拳的套路招式灵活
我我 我我我 我我我 活着生命就该完整渡过
我我 我我我 我我我 过错软弱从来不属于我
霍霍 霍霍霍 霍霍霍 我们精武出手无人能躲

吓 命有几回合 擂台等著 生死状 赢了什么 冷笑着
天下谁的 第一又如何 止干戈 我辈尚武德
我的 拳脚了得 却奈何 徒增虚名一个
江湖难测 谁是强者 谁争一统武林的资格

小城里岁月流过去 清澈的勇气
洗涤过的回忆 我记得你 骄傲的活下去

霍霍 霍霍霍 霍霍霍 霍家拳的套路招式灵活
我我 我我我 我我我 活着生命就该完整渡过
我我 我我我 我我我 过错软弱从来不属于我
霍霍 霍霍霍 霍霍霍 我们精武出手无人能躲

霍霍 霍霍霍 霍霍霍 霍家拳的套路招式灵活
我我 我我我 我我我 活着生命就该完整渡过
我我 我我我 我我我 过错软弱从来不属于我
霍霍 霍霍霍 霍霍霍 我们精武出手无人能躲

霍霍 霍霍霍 霍霍霍 霍家拳的套路招式灵活(小城里岁月流过去)
我我 我我我 我我我 活着生命就该完整渡过(清澈的勇气)
我我 我我我 我我我 过错软弱从来不属于我(洗涤过的回忆我记得你)
霍霍 霍霍霍 霍霍霍 我们精武出手无人能躲(骄傲的活下去~活下去~)'''

In [38]:
song_info.loc['霍元甲', 'lyric'] = '\n'.join([line for line in s.splitlines() if not line is ''])

In [39]:
song_info.head()

,DISPOSITION_attached_pic,TAG_AlbumArtist,TAG_ENCODEDBY,TAG_PLAY_COUNTER,TAG_PLAY_DATE,TAG_PLAY_TIME,TAG_WM/MediaPrimaryClassID,TAG_WM/Provider,TAG_WM/ProviderStyle,TAG_WM/Publisher,...,side_data_type,size,start_pts,start_time,time_base,width,lyricist,composer,arranger,lyric
song,,,,,,,,,,,,,,,,,,,,,
可爱女人,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Replay Gain,9601454,2255,0.025056,1/90000,445,徐若瑄,周杰伦,周杰伦,作词：徐若瑄\n作曲：周杰伦\n编曲：周杰伦\n想要有直升机 想要和妳飞到宇宙去\n想要和妳...
完美主义,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Replay Gain,9821927,2255,0.025056,1/90000,445,方文山,周杰伦,洪敬尧,作词：方文山\n作曲：周杰伦\n编曲：洪敬尧\n如果说怀疑 可以造句 如果说分离 能够翻译\...
星晴,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Replay Gain,10416474,2255,0.025056,1/90000,445,周杰伦,周杰伦,洪敬尧,作词：周杰伦\n作曲：周杰伦\n编曲：洪敬尧\n一步两步三步四步 望着天\n看星星 一颗两颗...
娘子,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Replay Gain,10880409,2255,0.025056,1/90000,445,方文山,周杰伦,周杰伦,作词：方文山\n作曲：周杰伦\n编曲：周杰伦\n娘子 娘子却依旧每日折一枝杨柳\n妳在那里 ...
斗牛,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Replay Gain,11197013,2255,0.025056,1/90000,445,方文山,周杰伦,洪敬尧,作词：方文山\n作曲：周杰伦\n编曲：洪敬尧\n要不要挑一下 啊 挑一下丫 随便阿˙\n有什...


In [40]:
new = song_info.copy()

In [41]:
new.columns

Index(['DISPOSITION_attached_pic', 'TAG_AlbumArtist', 'TAG_ENCODEDBY',
       'TAG_PLAY_COUNTER', 'TAG_PLAY_DATE', 'TAG_PLAY_TIME',
       'TAG_WM/MediaPrimaryClassID', 'TAG_WM/Provider', 'TAG_WM/ProviderStyle',
       'TAG_WM/Publisher', 'TAG_WM/UniqueFileIdentifier', 'TAG_WMFSDKNeeded',
       'TAG_WMFSDKVersion', 'TAG_WWW', 'TAG_Year', 'TAG_album',
       'TAG_album_artist', 'TAG_artist', 'TAG_composer', 'TAG_date',
       'TAG_encoder', 'TAG_genre', 'TAG_mp3gain_minmax', 'TAG_mp3gain_undo',
       'TAG_replaygain_track_gain', 'TAG_replaygain_track_peak', 'TAG_track',
       'avg_frame_rate', 'bit_rate', 'bits_per_raw_sample', 'channel_layout',
       'channels', 'chroma_location', 'codec_long_name', 'codec_name',
       'codec_tag', 'codec_tag_string', 'codec_time_base', 'codec_type',
       'coded_height', 'coded_width', 'color_primaries', 'color_range',
       'color_space', 'color_transfer', 'display_aspect_ratio', 'duration',
       'duration_ts', 'field_order', 'filename', 'fo

In [42]:
arranger = new.arranger
new.dropna(axis=1, thresh=len(new)*0.8, inplace=True)
new['arranger'] = arranger
new.shape

(180, 49)

In [43]:
# 去掉含有 20% 缺失值的列，除了 arranger
arranger = song_info.arranger
song_info.dropna(axis=1, thresh=len(song_info)*0.8, inplace=True)
song_info['arranger'] = arranger
song_info.shape

(180, 49)

In [44]:
# song_info.to_csv('all_songs_info.csv', index=False)

In [45]:
# def decode_gbk(item):
#     if not item is np.nan and isinstance(item, str):
#         try:
#             item = item.encode('latin1').decode('gbk')
#         except:
#             pass

In [46]:
# song_info = song_info.applymap(decode_gbk)

In [47]:
song_info.isnull().sum().sort_values(ascending=False)

arranger                    72
pix_fmt                     28
bits_per_raw_sample         28
chroma_location             28
coded_height                28
coded_width                 28
color_primaries             28
color_range                 28
color_space                 28
color_transfer              28
display_aspect_ratio        28
has_b_frames                28
height                      28
level                       28
field_order                 28
width                       28
refs                        28
sample_aspect_ratio         28
lyricist                    28
composer                    28
lyric                       27
bit_rate                     0
avg_frame_rate               0
channel_layout               0
codec_type                   0
channels                     0
TAG_artist                   0
TAG_album                    0
codec_long_name              0
codec_name                   0
codec_tag                    0
codec_tag_string             0
codec_ti

## 合并新的歌曲信息和专辑信息

In [48]:
song_info.TAG_album.unique()

array(['Jay', '七里香', '不能说的秘密', '依然范特西', '八度空间', '十一月的萧邦', '十二新作', '叶惠美',
       '哎呦，不错哦', '寻找周杰伦EP', '惊叹号！', '我很忙', '范特西', '跨时代', '霍元甲EP', '魔杰座',
       '黄金甲EP', '周杰伦的床边故事'], dtype=object)

In [49]:
with open('albums_info.json', encoding='utf8') as f:
    album_info = json.load(f)

In [50]:
album_info.keys()

dict_keys(['周杰伦的床边故事', '哎呦，不错哦', '十二新作', '惊叹号', '跨时代', '魔杰座', '我很忙(ON THE RUN)', '不能说的秘密', '依然范特西', '11月的萧邦', '寻找周杰伦', '七里香', '叶惠美', '八度空间', '范特西(Fantasy)', '杰伦(Jay)'])

In [51]:
album_info = pd.DataFrame.from_dict(album_info, orient='index')
album_info.head()

,date,distributor
11月的萧邦,200511,杰威尔音乐
七里香,200408,杰威尔音乐
不能说的秘密,200706,ALFA(亚津)
依然范特西,200609,杰威尔音乐
八度空间,200207,阿尔发/Bmg


In [52]:
album_info.index.unique()

Index(['11月的萧邦', '七里香', '不能说的秘密', '依然范特西', '八度空间', '十二新作', '叶惠美', '周杰伦的床边故事',
       '哎呦，不错哦', '寻找周杰伦', '惊叹号', '我很忙(ON THE RUN)', '杰伦(Jay)', '范特西(Fantasy)',
       '跨时代', '魔杰座'],
      dtype='object')

In [53]:
album_info.distributor.unique()

array(['杰威尔音乐', 'ALFA(亚津)', '阿尔发/Bmg', '阿尔发音乐', 'SONY MUSIC'],
      dtype=object)

In [54]:
song_info.TAG_album.unique()

array(['Jay', '七里香', '不能说的秘密', '依然范特西', '八度空间', '十一月的萧邦', '十二新作', '叶惠美',
       '哎呦，不错哦', '寻找周杰伦EP', '惊叹号！', '我很忙', '范特西', '跨时代', '霍元甲EP', '魔杰座',
       '黄金甲EP', '周杰伦的床边故事'], dtype=object)

In [55]:
album_info.rename(index={
    '寻找周杰伦': '寻找周杰伦EP',
    '我很忙(ON THE RUN)': '我很忙',
    '杰伦(Jay)': 'Jay',
    '范特西(Fantasy)': '范特西'
}, inplace=True)

album_info.loc['霍元甲EP'] = ['20060120', 'SONY MUSIC']
album_info.loc['黄金甲EP'] = ['20061207', 'SONY MUSIC']
album_info.loc['11月的萧邦', 'date'] = '20051101'
album_info.loc['七里香', 'date'] = '20040803'
album_info.loc['不能说的秘密', 'date'] = '20070813'
album_info.loc['依然范特西', 'date'] = '20060905'
album_info.loc['八度空间', 'date'] = '20020718'
album_info.loc['十二新作', 'date'] = '20121228'
album_info.loc['叶惠美', 'date'] = '20030729'
album_info.loc['周杰伦的床边故事', 'date'] = '20160624'
album_info.loc['哎呦，不错哦', 'date'] = '20141226'
album_info.loc['寻找周杰伦EP', 'date'] = '20031111'
album_info.loc['惊叹号', 'date'] = '20111111'
album_info.loc['我很忙', 'date'] = '20071101'
album_info.loc['Jay', 'date'] = '20001106'
album_info.loc['范特西', 'date'] = '20010914'
album_info.loc['跨时代', 'date'] = '20100518'
album_info.loc['魔杰座', 'date'] = '20081014'

album_info

,date,distributor
11月的萧邦,20051101,杰威尔音乐
七里香,20040803,杰威尔音乐
不能说的秘密,20070813,ALFA(亚津)
依然范特西,20060905,杰威尔音乐
八度空间,20020718,阿尔发/Bmg
十二新作,20121228,杰威尔音乐
叶惠美,20030729,杰威尔音乐
周杰伦的床边故事,20160624,杰威尔音乐
哎呦，不错哦,20141226,杰威尔音乐
寻找周杰伦EP,20031111,杰威尔音乐


In [56]:
song_info.TAG_album.replace({'惊叹号！': '惊叹号', '十一月的萧邦': '11月的萧邦'}, inplace=True)
song_info.TAG_album.unique()

array(['Jay', '七里香', '不能说的秘密', '依然范特西', '八度空间', '11月的萧邦', '十二新作', '叶惠美',
       '哎呦，不错哦', '寻找周杰伦EP', '惊叹号', '我很忙', '范特西', '跨时代', '霍元甲EP', '魔杰座',
       '黄金甲EP', '周杰伦的床边故事'], dtype=object)

In [57]:
song_info = pd.merge(song_info, album_info, how='left', left_on='TAG_album', right_index=True)
song_info.rename(columns={'date': 'release_date',
                          'TAG_album': 'album',
                          'TAG_artist': 'artist',
                          'DISPOSITION_attached_pic': 'attached_pic'}, inplace=True)
song_info.sample(5)

,attached_pic,album,artist,avg_frame_rate,bit_rate,bits_per_raw_sample,channel_layout,channels,chroma_location,codec_long_name,...,size,start_time,time_base,width,lyricist,composer,lyric,arranger,release_date,distributor
song,,,,,,,,,,,,,,,,,,,,,
上海 一九四三,1,范特西,周杰伦,0/0,322220,8,stereo,2,center,Motion JPEG,...,7887952,0.025056,1/90000,480,NaN,NaN,NaN,NaN,20010914,阿尔发/Bmg
给我一首歌的时间,1,魔杰座,周杰伦,0/0,321863,8,stereo,2,center,Motion JPEG,...,10203989,0.025056,1/90000,357,周杰伦,周杰伦,作词：周杰伦\n作曲：周杰伦\n编曲：林迈可\n监制：周杰伦\n雨淋湿了天空 毁得很讲究\n...,林迈可,20081014,杰威尔音乐
阳光宅男,1,我很忙,周杰伦,0/0,291120,8,stereo,2,center,Motion JPEG,...,8110485,0.025056,1/90000,352,方文山,周杰伦,作词：方文山\n作曲：周杰伦\n钥匙挂腰带 皮夹插 后面口袋 黑框的眼镜有 几千度 来海边穿...,NaN,20071101,杰威尔音乐
将军,1,七里香,周杰伦,0/0,320941,8,stereo,2,center,Motion JPEG,...,8096764,0,1/90000,143,黄俊郎,周杰伦,作词：黄俊郎\n作曲：周杰伦\n编曲：洪敬尧\n时间的箭头都指向你铩羽而归的地方\n你会前进...,洪敬尧,20040803,杰威尔音乐
First Kiss,1,不能说的秘密,周杰伦,0/0,324624,8,stereo,2,center,Motion JPEG,...,2903462,0,1/90000,240,NaN,NaN,NaN,NaN,20070813,ALFA(亚津)


In [58]:
song_info.release_date = pd.to_datetime(song_info.release_date, yearfirst=True)

In [59]:
song_info.loc[:, ['album', 'release_date', 'distributor']].sample(5)

,album,release_date,distributor
song,,,
自导自演,跨时代,2010-05-18,SONY MUSIC
公公偏头痛,十二新作,2012-12-28,杰威尔音乐
说好的幸福呢,魔杰座,2008-10-14,杰威尔音乐
斗琴,不能说的秘密,2007-08-13,ALFA(亚津)
扯,我很忙,2007-11-01,杰威尔音乐


In [60]:
song_info.to_csv('all_songs_info.csv', index=True)

In [61]:
# 保存专辑信息
album_info.date = pd.to_datetime(album_info.date, yearfirst=True)
album_info.index.name = 'album'
album_info.to_csv('album_info.csv', index=True)

In [62]:
song_info.columns

Index(['attached_pic', 'album', 'artist', 'avg_frame_rate', 'bit_rate',
       'bits_per_raw_sample', 'channel_layout', 'channels', 'chroma_location',
       'codec_long_name', 'codec_name', 'codec_tag', 'codec_tag_string',
       'codec_time_base', 'codec_type', 'coded_height', 'coded_width',
       'color_primaries', 'color_range', 'color_space', 'color_transfer',
       'display_aspect_ratio', 'duration', 'duration_ts', 'field_order',
       'filename', 'format_long_name', 'format_name', 'has_b_frames', 'height',
       'index', 'level', 'nb_streams', 'pix_fmt', 'probe_score', 'profile',
       'r_frame_rate', 'refs', 'sample_aspect_ratio', 'sample_fmt',
       'sample_rate', 'size', 'start_time', 'time_base', 'width', 'lyricist',
       'composer', 'lyric', 'arranger', 'release_date', 'distributor'],
      dtype='object')

In [63]:
song_info.index

Index(['可爱女人', '完美主义', '星晴', '娘子', '斗牛', '黑色幽默', '伊斯坦堡', '印地安老斑鸠', '龙卷风',
       '反方向的钟',
       ...
       'Now You See Me', '一点点', '前世情人', '告白气球', '土耳其冰淇淋', '床边故事', '爱情废柴', '英雄',
       '说走就走', '不该'],
      dtype='object', name='song', length=180)